# Init Spark

In [107]:
# import findspark
findspark.init()
findspark.find()
import pyspark

In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [7]:
sc

<SparkContext master=local appName=appName>

# Init Python

In [91]:
from IPython.display import display, HTML

In [11]:
import numpy as np
import pandas as pd

In [108]:
from pyspark.sql.types import Row, StructField, StringType, IntegerType, StructType

In [110]:
from multiprocessing import Pool as ProcessPool
from multiprocessing.dummy import Pool as ThreadPool

# Code

In [12]:
def func(data):
    df = pd.DataFrame({'A': [1,2], 'B': [2,3]})
    return [df.to_dict(orient='records')]
    
rdd.map(func).collect()

[[{'A': {0: 1, 1: 2}, 'B': {0: 2, 1: 3}}],
 [{'A': {0: 1, 1: 2}, 'B': {0: 2, 1: 3}}],
 [{'A': {0: 1, 1: 2}, 'B': {0: 2, 1: 3}}]]

In [46]:
def func(data):
    key, value = data[0], data[1]
    if key == 'A':
        df = pd.DataFrame({'A': [1,2], 'B': ['2','3']})
    else:
        df = pd.DataFrame({'B': ['2','3'], 'C': ['XXX', 'YYY'], 'D': ['ZZZ', 'ZZZ']})
    df['key'] = key
    return df.to_dict(orient='records')
    
schema = StructType([
    StructField('key', StringType(), nullable=True),
    StructField('A', IntegerType(), nullable=True),
    StructField('B', StringType(), nullable=True),
    StructField('C', StringType(), nullable=True)
])
sc.parallelize([('A', 1), ('B', 3)]).flatMap(func).collect()#.toDF(schema).toPandas()

[{'A': 1, 'B': '2', 'key': 'A'},
 {'A': 2, 'B': '3', 'key': 'A'},
 {'B': '2', 'C': 'XXX', 'D': 'ZZZ', 'key': 'B'},
 {'B': '3', 'C': 'YYY', 'D': 'ZZZ', 'key': 'B'}]

In [51]:
sc.parallelize([('A', 1), ('B', 3)]).flatMap(func).filter(lambda x: x['key'] == 'A').toDF().toPandas()

D:\3rd_party\spark-2.4.5-bin-hadoop2.7\python\pyspark\sql\session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


,A,B,key
0,1,2,A
1,2,3,A


In [116]:
def func(data):
    key, value = data[0], data[1]
    if key == 'A':
        df = pd.DataFrame({'A': [1,2], 'B': ['2','3']})
    elif key == 'B':
        df = pd.DataFrame({'A': [1,2], 'B': ['2','3'], 'C': ['A', 'V']})
    elif key == 'C':
        df = pd.DataFrame({'A': [1,2], 'B': ['2','3'], 'F': ['E', 'V']})
    elif key == 'D':
        df = pd.DataFrame({'A': [1,2], 'B': ['2','3'], 'G': ['P', 'U']})
    else:
        df = pd.DataFrame({'B': ['2','3'], 'C': ['XXX', 'YYY'], 'D': ['ZZZ', 'ZZZ']})
    df['key'] = key
    return [(key, df.to_dict(orient='records'))]

result_rdd = sc.parallelize([
    ('A', 1), 
    ('B', 3),
    ('C', 5),
    ('D', 7),
    ('E', 9),
]).flatMap(func).cache()
it = result_rdd.toLocalIterator()

In [117]:
keys = result_rdd.keys().collect()
for k in keys:
    sdf = result_rdd.filter(lambda x: x[0] == k).flatMap(lambda x: x[1]).toDF()
    sdf.write.csv('{}.csv'.format(k), header="true", mode="overwrite")
    #display(sdf.toPandas())

In [118]:
for k in result_rdd.keys().toLocalIterator():
    sdf = result_rdd.filter(lambda x: x[0] == k).flatMap(lambda x: x[1]).toDF()
    sdf.write.csv('{}.csv'.format(k), header="true", mode="overwrite")

In [ ]:
def par_func(k):
    print('{} Start'.format(k))
    sdf = result_rdd.filter(lambda x: x[0] == k).flatMap(lambda x: x[1]).toDF()
    sdf.write.csv('{}.csv'.format(k), header="true", mode="overwrite")
    print('{} End'.format(k))
    
pool = ThreadPool()
pool.map(par_func, keys)

In [70]:
for row in it:
    print(row)

('A', [{'A': 1, 'B': '2', 'key': 'A'}, {'A': 2, 'B': '3', 'key': 'A'}])
('B', [{'B': '2', 'C': 'XXX', 'D': 'ZZZ', 'key': 'B'}, {'B': '3', 'C': 'YYY', 'D': 'ZZZ', 'key': 'B'}])
